In [1]:
%cd ..
%pwd

/Users/ender_yang/Desktop/Spark/ds-ciss-policy-docs


'/Users/ender_yang/Desktop/Spark/ds-ciss-policy-docs'

In [2]:
from rank_bm25 import BM25Okapi
import jieba
import pandas as pd
from src.categories import get_l2_map
import numpy as np

In [3]:
df = pd.read_parquet('src/data/text_data.parquet')
df['decoded_column'] = df['content'].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)

In [4]:
df.head(10)

,file_name,content,decoded_column
0,中共天津市委教育工作委员会、天津市教育委员,b' \xe3\x80\x90\xe6\xb3\x95\xe5\xae\x9d\xe5...,【法宝引证码】 CLI.12.4720474\n原文链接：https://www.p...
1,南昌市发展和改革委员会关于九龙湖变电站立项,b' \xe3\x80\x90\xe6\xb3\x95\xe5\xae\x9d\xe5...,【法宝引证码】 CLI.12.2490990\n原文链接：https://www.p...
2,邯郸市人民政府办公厅关于印发邯郸市高危险性,b' \xe3\x80\x90\xe6\xb3\x95\xe5\xae\x9d\xe5...,【法宝引证码】 CLI.12.2662015\n原文链接：https://www.p...
3,成都市发展和改革委员会关于解放北路道路综合,b' \xe3\x80\x90\xe6\xb3\x95\xe5\xae\x9d\xe5...,【法宝引证码】 CLI.12.526157\n原文链接：https://www.pk...
4,海南省交通运输厅关于行政规范性文件清理结果,b' \xe3\x80\x90\xe6\xb3\x95\xe5\xae\x9d\xe5...,【法宝引证码】 CLI.12.5647676\n原文链接：https://www.p...
5,河北省物价局关于制定晶纯盐和竹盐等系列品种,b' \xe3\x80\x90\xe6\xb3\x95\xe5\xae\x9d\xe5...,【法宝引证码】 CLI.12.644477\n原文链接：https://www.pk...
6,天津市市场监管委关于印发《天津市企业经营范,b' \xe3\x80\x90\xe6\xb3\x95\xe5\xae\x9d\xe5...,【法宝引证码】 CLI.12.1592766\n原文链接：https://www.p...
7,南京市政府办公厅关于进一步加强和完善全市部,b' \xe3\x80\x90\xe6\xb3\x95\xe5\xae\x9d\xe5...,【法宝引证码】 CLI.12.1149805\n原文链接：https://www.p...
8,上海市卫生局关于印发《关于邀请市政府、市卫,b' \xe3\x80\x90\xe6\xb3\x95\xe5\xae\x9d\xe5...,【法宝引证码】 CLI.12.443753\n原文链接：https://www.pk...
9,苏州市应急管理局关于印发《苏州市应急管理局,b' \xe3\x80\x90\xe6\xb3\x95\xe5\xae\x9d\xe5...,【法宝引证码】 CLI.12.2817595\n原文链接：https://www.p...


In [5]:
topics = get_l2_map()
documents = df['file_name'].iloc[:100].tolist()
tokenized_docs = [list(jieba.cut(doc)) for doc in documents]
topics_titles = [l1 for l1 in topics.keys()]

Building prefix dict from /Users/ender_yang/opt/anaconda3/lib/python3.9/site-packages/jieba/dict.txt ...
Loading model from cache /var/folders/jj/4r8dblf53l9_ssdctfrstl480000gn/T/jieba.cache
Loading model cost 0.4353830814361572 seconds.
Prefix dict has been built succesfully.


In [6]:
topics_l2 = [l2 for l1 in topics.keys() for l2 in topics[l1]]

In [7]:
bm25 = BM25Okapi(tokenized_docs)
scores = np.ones((len(topics_l2), len(tokenized_docs)))
for topic in topics_l2:
    # Segment keywords
    segmented_keywords = list(jieba.cut(topic))
    # segmented_keywords = list(jieba.cut(' '.join(keywords)))

    segmented_keywords = [x for x in segmented_keywords if x != ' ' and x != '与' and x != '和']
    
    # Get BM25 scores for the segmented keywords against all documents
    score = bm25.get_scores(segmented_keywords)
    
    # aplly absolute value to the scores
    score = np.abs(score)
    topic_index = topics_l2.index(topic)
    scores[topic_index] = score
    

In [8]:
scores.shape

(92, 100)

In [13]:
scores[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [9]:
doc_category = np.argmax(scores, axis=0)

In [10]:
doc_category = [topics_l2[x] for x in doc_category]
doc_category

['教育',
 '农业',
 '农业',
 '金融综合',
 '交通运输',
 '农业',
 '企业',
 '农业',
 '农业',
 '农业',
 '农业',
 '农业',
 '农业',
 '农业',
 '农业',
 '农业',
 '经济审判',
 '农业',
 '农业',
 '卫生',
 '农业',
 '农业',
 '农业',
 '农业',
 '工业管理',
 '农业',
 '农业',
 '教育',
 '机关工作',
 '农业',
 '质量管理和监督',
 '农业',
 '农业',
 '经济审判',
 '人口与计划生育',
 '体育',
 '农业',
 '卫生',
 '质量管理和监督',
 '环境保护',
 '农业',
 '农业',
 '环境保护',
 '农业',
 '农业',
 '企业',
 '国有资产',
 '农业',
 '农业',
 '农业',
 '农业',
 '房地产',
 '人口与计划生育',
 '企业',
 '农业',
 '农业',
 '企业',
 '质量管理和监督',
 '农业',
 '社会监控',
 '农业',
 '农业',
 '经济审判',
 '农业',
 '农业',
 '文化',
 '农业',
 '农业',
 '交通运输',
 '农业',
 '农业',
 '农业',
 '农业',
 '农业',
 '农业',
 '体育',
 '金融综合',
 '农业',
 '农业',
 '质量管理和监督',
 '农业',
 '农业',
 '国家安全',
 '建设业',
 '农业',
 '农业',
 '农业',
 '农业',
 '农业',
 '质量管理和监督',
 '农业',
 '农业',
 '旅游',
 '工业管理',
 '农业',
 '卫生',
 '农业',
 '交通运输',
 '农业',
 '农业']